In [3]:
import cv2
import numpy as np
import tensorflow as tf
import mysql.connector
from datetime import datetime
import tkinter as tk
import os

# Load model .h5
model = tf.keras.models.load_model('absensi.h5')

# Directory to save captured images
images_dir = 'captured_images'
os.makedirs(images_dir, exist_ok=True)

# Load labels from labels.txt
with open('absensi_labels.txt', 'r') as f:
    labels = [line.strip() for line in f.readlines()]

# Connect to MySQL database `belajar`
conn = mysql.connector.connect(
    host='localhost',
    user='root',  # Sesuaikan dengan username MySQL Anda
    password='',  # Kosongkan jika tidak ada password
    database='belajar'  # Nama database yang digunakan
)
cursor = conn.cursor()

# Variable to store the detected class name and image
detected_name = None
detected_image = None

# Fungsi untuk update database
def update_database():
    global detected_name, detected_image
    if detected_name and detected_image is not None:
        now = datetime.now()
        timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
        status = "Hadir"
        
        # Save the image
        image_filename = f"{detected_name}_{now.strftime('%Y%m%d_%H%M%S')}.jpg"
        image_path = os.path.join(images_dir, image_filename)
        cv2.imwrite(image_path, detected_image)
        
        # Insert into MySQL database
        sql = "INSERT INTO face_recognition (timestamp, name, status, image_path) VALUES (%s, %s, %s, %s)"
        val = (timestamp, detected_name, status, image_path)
        cursor.execute(sql, val)
        conn.commit()
        print(f"Data {detected_name} dengan status {status} dan gambar {image_filename} telah disimpan ke database.")

# Fungsi untuk menangkap frame dan melakukan prediksi
def capture_frame():
    global detected_name, detected_image
    ret, frame = cap.read()
    if not ret:
        return

    # Preprocessing frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        roi_color = cv2.resize(roi_color, (224, 224))  # Sesuaikan dengan input size model Anda
        roi_color = np.expand_dims(roi_color, axis=0)
        
        # Predict class
        prediction = model.predict(roi_color)
        class_index = np.argmax(prediction, axis=1)
        
        # Use label from labels.txt
        detected_name = labels[class_index[0]]
        detected_image = frame[y:y+h, x:x+w]  # Save the detected face region
        
        # Display the results on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, detected_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    cv2.imshow('Face Recognition', frame)

# Fungsi untuk menangani loop GUI
def gui_loop():
    capture_frame()
    root.after(10, gui_loop)

# Setup GUI
root = tk.Tk()
root.title("Face Recognition")

# Tombol Accept
accept_button = tk.Button(root, text="Accept", command=update_database)
accept_button.pack()

# Mulai video capture
cap = cv2.VideoCapture(0)

# Start the GUI loop
root.after(10, gui_loop)
root.mainloop()

cap.release()
cv2.destroyAllWindows()

# Close MySQL connection
cursor.close()
conn.close()


1/1 [==============================] - 0s 40ms/step
Data Aditya dengan status Hadir dan gambar Aditya_20240829_025311.jpg telah disimpan ke database.
1/1 [==============================] - 0s 42ms/step
Data Aditya dengan status Hadir dan gambar Aditya_20240829_025341.jpg telah disimpan ke database.
1/1 [==============================] - 0s 39ms/step
Data Aditya dengan status Hadir dan gambar Aditya_20240829_025408.jpg telah disimpan ke database.
1/1 [==============================] - 0s 41ms/step
